<a href="https://colab.research.google.com/github/Balaji-github007/Python/blob/main/Python_Ergonomics_Calculation_Logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

# --- 1. Anthropometric Data (Simplified for demonstration) ---
# This data should ideally come from detailed anthropometric studies
# relevant to your target user population (e.g., Indian agricultural workers).
# Values are in cm.
# Percentiles (e.g., 5th, 50th, 95th) are important for inclusive design.
# For simplicity, we'll use a single "average" male figure.

# Source for conceptual values (adjust based on actual research for Indian population):
# These are illustrative and should be replaced with precise data.
# For example, "Anthropometric data of male agricultural workers of Gujarat (N=734)"
# or similar regional studies would be ideal.
ANTHROPOMETRY = {
    "average_male": {
        "sitting_height": 90.0,  # From seat to top of head
        "shoulder_height_sitting": 60.0, # From seat to shoulder
        "arm_length_shoulder_to_elbow": 30.0, # Upper arm
        "arm_length_elbow_to_wrist": 25.0,  # Forearm
        "hand_length_wrist_to_fingertip": 18.0, # Hand
        "torso_length": 50.0, # From hip to shoulder
        "leg_length_hip_to_knee": 45.0, # Thigh
        "leg_length_knee_to_ankle": 40.0, # Shin
        "foot_length": 25.0
    }
}

# --- 2. Human Model Representation (Simplified Kinematic Chains) ---
# Represents key body segments and their connections for reach analysis.
# Origin (0,0,0) is assumed to be at the Hip Joint (H-point).

class HumanModel:
    def __init__(self, body_type="average_male"):
        self.data = ANTHROPOMETRY.get(body_type, ANTHROPOMETRY["average_male"])

        # Define joint positions relative to the previous joint (conceptual for forward kinematics)
        # For a full simulator, inverse kinematics would be used to find joint angles for a target.
        self.joints = {
            "hip": [0, 0, 0], # Reference point, typically H-point of seat
            "shoulder": [0, self.data["torso_length"], 0], # Relative to hip
            "elbow": [self.data["arm_length_shoulder_to_elbow"], self.data["torso_length"], 0], # Relative to shoulder (simplified 2D for now)
            "wrist": [self.data["arm_length_shoulder_to_elbow"] + self.data["arm_length_elbow_to_wrist"], self.data["torso_length"], 0], # Relative to elbow
            "fingertip": [self.data["arm_length_shoulder_to_elbow"] + self.data["arm_length_elbow_to_wrist"] + self.data["hand_length_wrist_to_fingertip"], self.data["torso_length"], 0], # Relative to wrist
            "knee": [0, -self.data["leg_length_hip_to_knee"], 0], # Relative to hip
            "ankle": [0, -self.data["leg_length_hip_to_knee"] - self.data["leg_length_knee_to_ankle"], 0], # Relative to knee
            "foot_tip": [self.data["foot_length"], -self.data["leg_length_hip_to_knee"] - self.data["leg_length_knee_to_ankle"], 0] # Relative to ankle
        }

    def get_max_reach(self, limb="arm"):
        """Calculates the maximum linear reach of a limb from the hip origin."""
        if limb == "arm":
            return (self.data["arm_length_shoulder_to_elbow"] +
                    self.data["arm_length_elbow_to_wrist"] +
                    self.data["hand_length_wrist_to_fingertip"])
        elif limb == "leg":
            return (self.data["leg_length_hip_to_knee"] +
                    self.data["leg_length_knee_to_ankle"] +
                    self.data["foot_length"])
        return 0

    def check_reach(self, target_coords, origin_coords, limb="arm"):
        """
        Checks if a target is reachable by a specified limb.
        target_coords: [x, y, z] of the control
        origin_coords: [x, y, z] of the limb's base joint (e.g., shoulder for arm, hip for leg)
        """
        distance = math.sqrt(
            (target_coords[0] - origin_coords[0])**2 +
            (target_coords[1] - origin_coords[1])**2 +
            (target_coords[2] - origin_coords[2])**2
        )
        max_reach = self.get_max_reach(limb)

        if distance <= max_reach:
            return True, distance, "Reachable"
        else:
            return False, distance, "Out of reach"

    def evaluate_posture(self, joint_angles):
        """
        Evaluates a simplified posture based on joint angles.
        (This would be more complex with actual inverse kinematics and ergonomic guidelines)
        joint_angles: A dictionary of joint angles (e.g., {"elbow": 120, "shoulder": 45})
        Returns a comfort score or warnings.
        """
        comfort_score = 100 # Start with perfect comfort

        # Example: Simple rules for arm posture
        if "elbow" in joint_angles:
            elbow_angle = joint_angles["elbow"]
            if not (90 <= elbow_angle <= 160): # Ideal range for many tasks
                comfort_score -= 10
                print(f"Warning: Elbow angle {elbow_angle}° is outside optimal range (90-160°).")
        if "shoulder" in joint_angles:
            shoulder_angle = joint_angles["shoulder"]
            if not (-30 <= shoulder_angle <= 60): # Example range for forward/upward reach
                comfort_score -= 15
                print(f"Warning: Shoulder angle {shoulder_angle}° is outside optimal range (-30-60°).")

        # More rules for other joints, twisting, etc.
        # Ideally, this would use established ergonomic standards (e.g., ISO, SAE guidelines).

        return max(0, comfort_score), "Posture evaluated"

# --- Example Usage (simulating a backend calculation) ---
if __name__ == "__main__":
    operator = HumanModel("average_male")

    print(f"Max arm reach: {operator.get_max_reach('arm')} cm")
    print(f"Max leg reach: {operator.get_max_reach('leg')} cm")

    # Simulate a control's position relative to the hip (origin)
    # Example: A steering wheel at [x, y, z] relative to hip
    steering_wheel_pos = [40, 50, 20] # 40cm forward, 50cm up (from shoulder), 20cm to the side
    pedal_pos = [60, -80, 10] # 60cm forward, 80cm down, 10cm to the side

    # Check reach for steering wheel
    reachable_sw, dist_sw, status_sw = operator.check_reach(steering_wheel_pos, operator.joints["shoulder"], limb="arm")
    print(f"\nSteering wheel at {steering_wheel_pos} cm:")
    print(f"  Reach Status: {status_sw} (Distance: {dist_sw:.2f} cm)")

    # Check reach for pedal
    reachable_pedal, dist_pedal, status_pedal = operator.check_reach(pedal_pos, operator.joints["hip"], limb="leg")
    print(f"Pedal at {pedal_pos} cm:")
    print(f"  Reach Status: {status_pedal} (Distance: {dist_pedal:.2f} cm)")

    # Simulate posture evaluation (requires actual joint angles from inverse kinematics)
    # For this example, we'll use hypothetical angles.
    hypothetical_arm_angles = {"elbow": 100, "shoulder": 40}
    comfort, msg = operator.evaluate_posture(hypothetical_arm_angles)
    print(f"\nHypothetical Arm Posture Comfort: {comfort} (Message: {msg})")

    hypothetical_bad_arm_angles = {"elbow": 170, "shoulder": 80}
    comfort_bad, msg_bad = operator.evaluate_posture(hypothetical_bad_arm_angles)
    print(f"Hypothetical Bad Arm Posture Comfort: {comfort_bad} (Message: {msg_bad})")